In [24]:
import pickle
import numpy as np
from keyword_spotting import predictions as posterior_handling

In [26]:
FILE='../models.pickle'
FILE='/Users/emi/cnn_visiontransformer.pkl'

In [27]:
with open(FILE, 'rb') as f:
    config, time, predictions = pickle.load(f)

In [28]:
config

{'dataset': {'path': '/home/luciano/speech1'},
 'model': {'path': '/home/luciano/models/cnn_visiontransformer_model',
  'name': 'cnn_visiontransformer'},
 'train': {'batch_size': 1024, 'epochs': 14},
 'output_path': '/home/luciano/models/cnn_visiontransformer.pkl'}

In [29]:
time

33724.72209906578

In [30]:
x0 =1
keyword, real_label, labels_posteriors = predictions[x0]

In [44]:
preds = []
trues = []
for _, real_label, labels_posterios in predictions:
    preds.extend((np.ones(labels_posterios.shape[0])*real_label).astype('int'))
    trues.extend(np.argmax(labels_posterios,axis=1))

In [57]:
q = np.array(trues) != 11
tt = np.squeeze(np.array(trues)[q])
pp = np.squeeze(np.array(preds)[q])


In [59]:
accuracy_score(tt, pp)

0.6570391770559972

In [9]:
real_label

10

In [10]:
np.argmax(labels_posteriors, 1)

array([ 9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  1,  9,  3,  3,
        3,  9,  9,  3,  3,  9,  1,  2,  3,  1,  1,  2,  3,  1,  1,  2,  2,
       11,  2,  2,  2, 11,  2,  2,  2,  2,  2,  6,  1,  1,  1,  1,  1,  3,
        2,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2])

In [11]:
keyword

'/home/luciano/speech1/go/3b4f8f24_nohash_4.wav'

In [12]:
real_label

10

In [13]:
len(labels_posteriors)

66

66 frames para el audio d21fd169_nohash_0.wav

In [14]:
type(labels_posteriors[0])

numpy.ndarray

In [15]:
labels_posteriors[0:3, 1]
# asi selecciono las probabilidades para el label 1 para los frames 0,1 y 2s

array([0.18356495, 0.19209503, 0.19428451], dtype=float32)

las probabilidades del frame 0 del primero audio, una prob para cada label, 12 en total. para la primer ventanita, da prioridad mas alta sobre la ultima clase la 11

## roberto handling

Suppose $p_{ij}$ is the neural network posterior for the $i$th label and the $j$th frame $x_j$, where $i$ takes values between $0, 1, ..., n − 1$, with $n$ the number of total labels and 0 the label for non-keyword

### Posterior smoothing



In [16]:
import numpy as np

In [17]:
smooth_w = 2
posteriors_label1=labels_posteriors[:, 1]
smooth_posteriors_label1 = posterior_handling.posterior_smoothing(posteriors_label1, smooth_w)

In [18]:
print(posteriors_label1[0:5])
print(smooth_posteriors_label1[0:5])

[0.18356495 0.19209503 0.19428451 0.16486883 0.11740703]
[0.18356495, 0.18782999, 0.19318977, 0.17957667, 0.14113793]


In [19]:
smooth_posteriors = np.zeros(labels_posteriors.shape)

for label in range(labels_posteriors.shape[1]):
    
    posteriors_label=labels_posteriors[:, label]
    smooth_posteriors[:, label] = posterior_handling.posterior_smoothing(posteriors_label, smooth_w)

Para cada frame del audio 1 voy a calcular la confidence

### Confidence

In [20]:
max_w = 5
confidences = posterior_handling.confidence(smooth_posteriors, max_w )

(66, 12)


In [21]:
# Get the frame with the highest probability
frame_best_confidence = np.argmax(confidences)

# For the frame with highest confidence we obtain the label with the highest posterior
np.argmax(labels_posteriors[frame_best_confidence])

2

In [22]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Get the prediction label for every wav

Evaluar las predicciones para distintos valores de ventana de smooth y max windows

In [23]:
from tqdm.auto import tqdm
smooth_w = 30
max_w = 100
total_labels = predictions[0][2].shape[1]
y_true = []
y_pred = []
for keyword, real_label, labels_posteriors in tqdm(predictions):
    
    y_true.append(real_label)
    
    # Obtain the prediction with handling posterior
    predicted_label = posterior_handling.get_predictions(labels_posteriors, total_labels, smooth_w, max_w)
    print(predicted_label)
    y_pred.append(predicted_label)
    try:
        print(accuracy_score(y_true, y_pred))
    except:
        pass

  0%|          | 0/4363 [00:00<?, ?it/s]

(66, 12)
[0.06242582 0.06196545 0.06151495 0.06101722 0.06048293 0.05987169
 0.0591421  0.05840432 0.05810093 0.05782498]
[4]
0.0
(66, 12)
[0.08665083 0.08651334 0.08622078 0.08599621 0.08585139 0.08574163
 0.08549128 0.08528479 0.08505612 0.08485445]
[2]
0.0
(66, 12)
[0.07432919 0.07421155 0.07391205 0.07330205 0.07269597 0.07269597
 0.07269597 0.07269597 0.07269597 0.07269597]
[0]
0.3333333333333333
(60, 12)
[0.0635897  0.0635897  0.0635897  0.06358673 0.06358547 0.06358159
 0.06357299 0.06356636 0.06355427 0.06353791]
[8]
0.5
(66, 12)
[0.03700214 0.03700214 0.03700214 0.03700214 0.03700214 0.03700214
 0.03700214 0.03700214 0.03700214 0.03700214]
[8]
0.6
(66, 12)
[0.09576183 0.09553214 0.09529289 0.09504055 0.09477569 0.09415615
 0.0936816  0.09358211 0.09357224 0.09357224]
[6]
0.6666666666666666
(66, 12)
[0.03632535 0.03598143 0.03593618 0.03586074 0.03577044 0.03568306
 0.03558605 0.03547592 0.03535872 0.03523822]
[9]
0.7142857142857143
(66, 12)
[0.0756349  0.07531323 0.07500776 0.

(66, 12)
[0.07284749 0.07245544 0.07245544 0.07243747 0.07232052 0.0722271
 0.07207776 0.07194825 0.07181814 0.07174167]
[3]
0.47692307692307695
(66, 12)
[0.10994568 0.10968863 0.10936615 0.10914509 0.10875003 0.10834354
 0.10795287 0.10794833 0.1078177  0.1078177 ]
[7]
0.4696969696969697
(66, 12)
[0.05642947 0.05584624 0.05514827 0.05455968 0.05376284 0.05322468
 0.05261591 0.05205531 0.05138165 0.05075155]
[6]
0.47761194029850745
(66, 12)
[0.0210463  0.02104558 0.02104459 0.02104383 0.02104286 0.02104202
 0.02104059 0.02103853 0.02103692 0.02103479]
[0]
0.4852941176470588
(66, 12)
[0.02874403 0.02856705 0.0284321  0.02823106 0.0281148  0.02785024
 0.02760893 0.02744959 0.02731449 0.0269994 ]
[2]
0.4927536231884058
(66, 12)
[0.0455903  0.04348234 0.04159636 0.03983345 0.03842878 0.03670637
 0.03509507 0.03362402 0.03260172 0.03143775]
[6]
0.5
(66, 12)
[0.02676601 0.02657464 0.02619789 0.02520795 0.02506475 0.02498543
 0.02495897 0.02495897 0.02495897 0.02495897]
[6]
0.5070422535211268

(66, 12)
[0.07186768 0.07168848 0.0716614  0.07135407 0.07082084 0.0704543
 0.07035169 0.07009874 0.06981262 0.06952395]
[10]
0.5915492957746479
(66, 12)
[0.05091676 0.05023013 0.04942079 0.04900392 0.04831986 0.04700669
 0.04505327 0.043407   0.04152231 0.0389918 ]
[0]
0.5944055944055944
(6091, 12)
[0.00034184 0.00034184 0.00034184 0.00034184 0.00034184 0.00034184
 0.00034184 0.00034184 0.00034184 0.00034184]
[11]
0.5972222222222222
(66, 12)
[0.07900895 0.07857768 0.07808579 0.0775827  0.07705951 0.07652925
 0.07632635 0.07632635 0.07632635 0.07632635]
[1]
0.6
(66, 12)
[0.08379102 0.08374418 0.08369525 0.08364107 0.08355235 0.08343234
 0.08329932 0.08315495 0.08297588 0.08277308]
[5]
0.6027397260273972
(66, 12)
[0.08549997 0.08536024 0.08530713 0.08530334 0.08520276 0.08510964
 0.08493254 0.08475491 0.0841215  0.0836269 ]
[9]
0.5986394557823129
(66, 12)
[0.08792484 0.08715971 0.08607499 0.08514937 0.08391251 0.08294084
 0.08056805 0.07936403 0.07742874 0.07555318]
[2]
0.59459459459459

(66, 12)
[0.00937215 0.0085497  0.00806978 0.00718604 0.0058231  0.0051495
 0.00480174 0.00440616 0.00349721 0.00303759]
[0]
0.5897435897435898
(66, 12)
[0.05395655 0.05395655 0.05395655 0.05395655 0.05395655 0.05394188
 0.05388607 0.05381218 0.05372166 0.05361835]
[9]
0.5914893617021276
(66, 12)
[0.06088321 0.05987276 0.05876816 0.05804704 0.05785837 0.05772901
 0.05751123 0.05713361 0.0568184  0.05654671]
[4]
0.5889830508474576
(66, 12)
[0.07270661 0.07146399 0.07019105 0.06961682 0.06961682 0.06961682
 0.06961682 0.06961682 0.06961682 0.06961682]
[7]
0.5907172995780591
(66, 12)
[0.11567462 0.11567153 0.11553646 0.11529249 0.11516575 0.11513675
 0.11450569 0.11369757 0.11287971 0.11218744]
[11]
0.5882352941176471
(66, 12)
[0.04298712 0.04205673 0.04104258 0.03973505 0.03847374 0.0370505
 0.03563647 0.03408553 0.03272057 0.03111174]
[2]
0.5857740585774058
(66, 12)
[0.06266193 0.06193876 0.06125228 0.06050435 0.05982272 0.05881591
 0.05795968 0.0570574  0.05614192 0.05495195]
[4]
0.587

...

(9484, 12)
[0.00074477 0.00074477 0.00074477 0.00074477 0.00074477 0.00074477
 0.00074477 0.00074477 0.00074477 0.00074477]
[11]
0.6038251366120219
(60, 12)
[0.07176728 0.07147454 0.0712217  0.0708821  0.07080523 0.07080523
 0.07080523 0.07080523 0.07080523 0.07080523]
[2]
0.6049046321525886
(66, 12)
[0.05910877 0.0564869  0.05051363 0.04924554 0.04185205 0.03978888
 0.03688616 0.03621338 0.03572272 0.03533986]
[2]
0.6059782608695652
(66, 12)
[0.04304708 0.04282056 0.0425804  0.0423342  0.04200662 0.04169554
 0.04135766 0.0409819  0.04047465 0.04011513]
[2]
0.6043360433604336
(66, 12)
[0.01207105 0.01082043 0.01016255 0.00935488 0.00864927 0.00787679
 0.00738591 0.00672745 0.00563238 0.0044202 ]
[0]
0.6054054054054054
(66, 12)
[0.10502988 0.10396589 0.10305522 0.10257234 0.10233313 0.10206022
 0.10162379 0.10136542 0.10136542 0.10136542]
[9]
0.6037735849056604
(66, 12)
[0.07287523 0.07287523 0.07284435 0.07279351 0.07270944 0.07268459
 0.07268255 0.07268255 0.07268255 0.07268255]
[9]
0

(6091, 12)
[0.00034184 0.00034184 0.00034184 0.00034184 0.00034184 0.00034184
 0.00034184 0.00034184 0.00034184 0.00034184]
[11]
0.6107226107226107
(9484, 12)
[0.00074477 0.00074477 0.00074477 0.00074477 0.00074477 0.00074477
 0.00074477 0.00074477 0.00074477 0.00074477]
[11]
0.6116279069767442
(66, 12)
[0.02305423 0.02266528 0.02234116 0.02195145 0.02146425 0.02115263
 0.02072117 0.02039281 0.01994583 0.01974295]
[2]
0.6125290023201856
(66, 12)
[0.03689393 0.03633535 0.03572067 0.03516922 0.03451521 0.03390107
 0.03316877 0.03242891 0.03136693 0.03007125]
[1]
0.6111111111111112
(66, 12)
[0.10002077 0.0994522  0.09905048 0.0980809  0.09681512 0.09604639
 0.09496665 0.09382641 0.09284557 0.09201579]
[1]
0.6120092378752887
(56, 12)
[0.12638881 0.1263165  0.12614233 0.12573663 0.12524012 0.12459456
 0.12406005 0.12350477 0.12281658 0.12211484]
[4]
0.6105990783410138
(66, 12)
[0.0443599  0.04412296 0.04389039 0.04360353 0.04326516 0.04284035
 0.04230168 0.0416101  0.04100456 0.04027256]
[1

(5966, 12)
[0.00052919 0.00052919 0.00052919 0.00052919 0.00052919 0.00052919
 0.00052919 0.00052919 0.00052919 0.00052919]
[11]
0.594704684317719
(66, 12)
[0.0808988  0.08026178 0.07943379 0.07870638 0.0780845  0.07728747
 0.07656958 0.07582717 0.0752323  0.07476761]
[9]
0.5934959349593496
(66, 12)
[0.11405863 0.11357745 0.11307854 0.11269062 0.11199637 0.11085066
 0.10949757 0.10824511 0.10671383 0.10477433]
[2]
0.592292089249493
(66, 12)
[0.06420631 0.0639625  0.06373097 0.06351267 0.06330141 0.06302481
 0.06276104 0.0625171  0.06230416 0.06205588]
[7]
0.5910931174089069
(66, 12)
[0.08001093 0.0790843  0.07843075 0.07840522 0.06914772 0.06914772
 0.06914772 0.06914772 0.06914772 0.06914653]
[4]
0.591919191919192
(66, 12)
[0.07339548 0.07289764 0.07246072 0.07246072 0.07246072 0.07246072
 0.07246072 0.07246072 0.07246072 0.07246072]
[6]
0.592741935483871
(66, 12)
[0.01889126 0.01823995 0.017316   0.01687413 0.01636935 0.01607794
 0.01563165 0.01513099 0.01442932 0.01390022]
[3]
0.593

(6147, 12)
[0.00048302 0.00048302 0.00048302 0.00048302 0.00048302 0.00048302
 0.00048302 0.00048302 0.00048302 0.00048302]
[11]
0.5866425992779783
(66, 12)
[0.04882724 0.04882724 0.04882724 0.04882724 0.04882724 0.04882724
 0.04882724 0.04882724 0.04882724 0.04882724]
[11]
0.5855855855855856
(66, 12)
[0.07892258 0.07892258 0.07892258 0.07892258 0.07892258 0.07892258
 0.07892258 0.07892258 0.07892258 0.07892258]
[9]
0.5863309352517986
(66, 12)
[0.04439203 0.04108073 0.03828462 0.0353334  0.03279646 0.02947309
 0.02635015 0.02278032 0.01925248 0.01454569]
[9]
0.585278276481149
(66, 12)
[0.04873924 0.04792538 0.04713922 0.04648254 0.04570311 0.0449702
 0.04387636 0.04276253 0.04152077 0.03996732]
[7]
0.5860215053763441
(66, 12)
[0.08541646 0.084762   0.08417794 0.08350529 0.08250265 0.08140536
 0.08048297 0.07966138 0.07919688 0.07919688]
[8]
0.5849731663685152
(66, 12)
[0.10401416 0.10323047 0.10235742 0.10147759 0.10053708 0.09956439
 0.09851531 0.09750647 0.09634279 0.09530632]
[2]
0.

KeyboardInterrupt: 

In [ ]:
smooth_posteriors.shape

In [ ]:
accuracy_score(y_true, y_pred)

In [ ]:
print(y_true[0:50])

In [ ]:
print(y_pred[0:50])

In [ ]:
confusion_matrix(y_true, y_pred)